In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv 
from langchain_core.prompts import PromptTemplate
import os 

os.environ["HF_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")
os.environ["HF_HOME"] = os.getenv("HF_HOME")

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation"
)
model = ChatHuggingFace(llm = llm)

d:\Code\Python\genai-projects\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### StrOutputParser

In [2]:
# 1st prompt for Detailed report 
template1 = PromptTemplate(
    template="Write a detailed report on {topic}",
    input_variables=["topic"]
)

# 2nd prompt for Summary
template2 = PromptTemplate(
    template="Write a 5 line summary on the following text. /n {text}",
    input_variables=["text"]
)

In [3]:
# Without parser approach

prompt1 = template1.invoke({ "topic": "Black hole" })
result = model.invoke(prompt1)
prompt2 = template2.invoke({ "text": result.content })
final_result = model.invoke(prompt2)

final_result

AIMessage(content='Here is a 5-line summary of the provided text:\n\nBlack holes are regions in space where gravity is so strong that nothing, including light, can escape. They are formed when a massive star collapses, and their properties and behavior are still not fully understood. Black holes have several unique properties, including an event horizon, singularity, and gravitational pull. There are four types of black holes, each with different properties and behaviors, and scientists continue to study them using various methods. Understanding black holes is crucial for understanding the universe and their impact on galaxy evolution and the behavior of matter and energy.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 1189, 'total_tokens': 1309}, 'model_name': 'meta-llama/Llama-3.2-3B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--28fd159d-fe89-4283-9582-748f035e9c2c-0', usage_metadata=

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = template1 | model | parser | template2 | model | parser

chain.get_graph().print_ascii()

result = chain.invoke({ "topic": "Black hole" })

print(result)

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
   +-----------------+     
   | ChatHuggingFace |     
   +-----------------+     
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *       

### JsonOutputParser

In [5]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

template = PromptTemplate(
    template="Give me 5 facts about {topic} \n {format}",
    input_variables=["topic"],
    partial_variables={
        "format": parser.get_format_instructions()
    }
)
chain = template | model | parser

result = chain.invoke({ "topic": "Black hole" })
print(result)

{'fact1': 'A black hole is a region in space where the gravitational pull is so strong that nothing, including light, can escape.', 'fact2': 'Black holes are formed when a massive star collapses in on itself and its gravity becomes so strong that it warps the fabric of spacetime.', 'fact3': "The point of no return around a black hole is called the event horizon, and once something crosses the event horizon, it is trapped by the black hole's gravity forever.", 'fact4': 'Black holes come in different sizes, ranging from small, stellar-mass black holes formed from the collapse of individual stars, to supermassive black holes found at the centers of galaxies, which can have masses billions of times that of the sun.', 'fact5': "The term 'black hole' was coined by American physicist John Wheeler in 1964, and it is derived from the idea that the gravity of the black hole is so strong that it 'swallows up' all light and matter that gets too close."}


### StructuredOutputParser

In [6]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

schema = [
    ResponseSchema(name="fact_1", description="Fact 1 about the topic"),
    ResponseSchema(name="fact_2", description="Fact 2 about the topic"),
    ResponseSchema(name="fact_3", description="Fact 3 about the topic")
]
parser = StructuredOutputParser.from_response_schemas(schema)

In [7]:
template = PromptTemplate(
    template="Give 3 facts about the topic {topic} \n {format_instructions}",
    input_variables=["topic"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
chain = template | model | parser
result = chain.invoke({"topic": "Cricket"})
print(result)

{'fact_1': 'Cricket is one of the most popular sports in the world, played in over 100 countries with a massive following of over 2.5 billion fans.', 'fact_2': 'The first recorded game of cricket was played in 1598 at the Artillery Ground in London, England, and the game has since evolved into various formats including Test cricket, One Day Internationals (ODIs), and Twenty20 (T20).', 'fact_3': 'The longest cricket match was played between Australia and South Africa in 1939, lasting for 5 days and 18 hours, with Australia winning by 4 wickets.'}


### PydanticOutputParser

In [10]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str = Field(description="Name of the person")
    age: int = Field(gt=18, description="Age of the person")
    city: str = Field(description="City where the person lives")

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template="Generate the name, age, city of a fictional {place} person \n {format_instruction}",
    input_variables=["place"],
    partial_variables={
        "format_instruction": parser.get_format_instructions()
    }
)

chain = template | model | parser 
final_result = chain.invoke({ "place": "Indian" })
print(final_result)

OutputParserException: Invalid json output: Here's a Python script that generates a JSON instance conforming to the provided schema:

```python
import json
import random

# Define the schema
schema = {
    "properties": {
        "name": {
            "description": "Name of the person",
            "title": "Name",
            "type": "string"
        },
        "age": {
            "description": "Age of the person",
            "exclusiveMinimum": 18,
            "title": "Age",
            "type": "integer"
        },
        "city": {
            "description": "City where the person lives",
            "title": "City",
            "type": "string"
        }
    },
    "required": ["name", "age", "city"]
}

# Function to generate a random name
def generate_name():
    first_names = ["Rahul", "Aryan", "Karan", "Amit", "Rohan"]
    last_names = ["Pandey", "Sharma", "Singh", "Gupta", "Chaudhary"]
    return f"{random.choice(first_names)} {random.choice(last_names)}"

# Function to ge